## Collecting protocol fees across Balancer core pools on all networks
- Spreadsheet as reference: https://docs.google.com/spreadsheets/d/1xwUPpbYq7woVOU9vQ8EB8MY75I-1mauTLyDVwvKUDKo/edit#gid=0
- Collab: https://colab.research.google.com/drive/1vKCvcV5mkL1zwW3565kLSGkBEbt8NsoB?usp=sharing


In [1]:
import datetime
import os

from dotenv import load_dotenv
from web3 import Web3
from web3.middleware import geth_poa_middleware

from notebooks import get_twap_bpt_price
from notebooks import get_block_by_ts
from notebooks.constants import ARB_CORE_POOLS
from notebooks.constants import MAINNET_CORE_POOLS
from notebooks.constants import POLYGON_CORE_POOLS

load_dotenv()
arb_web3 = Web3(Web3.HTTPProvider(os.environ["ARBNODEURL"]))
eth_web3 = Web3(Web3.HTTPProvider(os.environ["ETHNODEURL"]))
poly_web3 = Web3(Web3.HTTPProvider("https://polygon-rpc.com"))
poly_web3.middleware_onion.inject(geth_poa_middleware, layer=0)

# ARBITRUM
# arb_block_now = arb_web3.eth.block_number - 1000
timestamp_now = 1692316800
arb_block = get_block_by_ts(timestamp_now, "arbitrum")  # 18 August 2023
# Given Arb block time, we want to look back 2 weeks:
arb_timestamp_2_weeks_ago = timestamp_now - (2 * 7 * 24 * 60 * 60)
arb_block_2_weeks_ago = get_block_by_ts(arb_timestamp_2_weeks_ago, "arbitrum")
# Convert to datetime:
arb_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
arb_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(arb_timestamp_2_weeks_ago)
# MAINNET
mainnet_block = get_block_by_ts(timestamp_now, "mainnet")
# Given mainnet block time, we want to look back 2 weeks:
mainnet_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
mainnet_block_2_weeks_ago = get_block_by_ts(mainnet_timestamp_2_weeks_ago, "mainnet")
# Convert to datetime:
mainnet_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
mainnet_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(mainnet_timestamp_2_weeks_ago)

# POLYGON
poly_block = get_block_by_ts(timestamp_now, "polygon")
# Given polygon block time, we want to look back 2 weeks:
poly_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
poly_block_2_weeks_ago = get_block_by_ts(poly_timestamp_2_weeks_ago, "polygon")
# Convert to datetime:
poly_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
poly_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(poly_timestamp_2_weeks_ago)

print("ARBITRUM")
arb_bpt_twap_prices = {}
for arb_pool in ARB_CORE_POOLS:
    arb_bpt_twap_prices[arb_pool] = get_twap_bpt_price(arb_pool, 'arbitrum', arb_web3, start_date=arb_datetime_now.date(),
                                                       block_number=arb_block)
    print(arb_bpt_twap_prices[arb_pool])

print("MAINNET")
mainnet_bpt_twap_prices = {}
for mainnet_pool in MAINNET_CORE_POOLS:
    mainnet_bpt_twap_prices[mainnet_pool] = get_twap_bpt_price(mainnet_pool, 'mainnet', eth_web3,
                                                               start_date=mainnet_datetime_now.date(),
                                                               block_number=mainnet_block)
    print(mainnet_bpt_twap_prices[mainnet_pool])

print("POLYGON")
polygon_bpt_twap_prices = {}
for polygon_pool in POLYGON_CORE_POOLS:
    polygon_bpt_twap_prices[polygon_pool] = get_twap_bpt_price(polygon_pool, 'polygon', poly_web3,
                                                               start_date=poly_datetime_now.date(), block_number=poly_block)
    print(polygon_bpt_twap_prices[polygon_pool])


ARBITRUM
1833.451852052343848962140634
1842.209235229654557639362022
1834.091377165366659324871867
1.000520816308704170788477865
1.373806462439175997013422238
1832.497120530902215323283611
1.000395968349039228383157872
MAINNET
1831.173736601367607918698024
1879.372713804580062835519769
1819.470889750963667126670163
1831.074797956762334441595632
1830.577686949676287358700022
28.70541367671049478113772939
1847.755351730498445845029905
0.9812378645358918062838625662
1838.469578858441677257480486
60.78884759087628281622369958
66.87150794940494276939754347
231.1831220542486363085535130
0.7977727270803851051516774764
1841.843380948226418306261490
11.56327451907746932140415258
POLYGON
0.9996100241883402138399179987
0.9988396935020339437258447976
0.6563199419006072856049719627
0.6561457562299947277772776168
1831.097351676973776713782586


In [2]:
# Query:
POOLS_SNAPSHOTS_QUERY = """
{{
  poolSnapshots(
    first: {first}
    skip: {skip}
    orderBy: timestamp
    orderDirection: desc
    block: {{ number: {block} }}
    where: {{ protocolFee_not: null }}
  ) {{
    pool {{
      address
      id
      symbol
      totalProtocolFeePaidInBPT
      tokens {{
        symbol
        address
        paidProtocolFees
      }}
    }}
    timestamp
    protocolFee
    swapFees
    swapVolume
    liquidity
  }}
}}
"""

## Fetching data from the Balancer subgraphs

In [3]:
from notebooks.constants import POLYGON_BALANCER_GRAPH_URL
from notebooks.constants import ARB_BALANCER_GRAPH_URL
from notebooks.constants import MAINNET_BALANCER_GRAPH_URL
from typing import Dict
from typing import List
from typing import Optional

from gql import Client
from gql import gql
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd


# Fetch all the data from the balancer subgraph
def make_gql_client(url: str) -> Optional[Client]:
    transport = RequestsHTTPTransport(url=url, retries=3)
    return Client(
        transport=transport, fetch_schema_from_transport=True, execute_timeout=60
    )


def get_balancer_pool_snapshots(block: int, graph_url: str) -> Optional[List[Dict]]:
    client = make_gql_client(graph_url)
    all_pools = []
    limit = 1000
    offset = 0
    while True:
        result = client.execute(
            gql(POOLS_SNAPSHOTS_QUERY.format(first=limit, skip=offset, block=block)))
        all_pools.extend(result['poolSnapshots'])
        offset += limit
        if offset >= 5000:
            break
        if len(result['poolSnapshots']) < limit - 1:
            break
    return all_pools


arbi_pool_snapshots_now = get_balancer_pool_snapshots(arb_block, ARB_BALANCER_GRAPH_URL)
arbi_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(arb_block_2_weeks_ago), ARB_BALANCER_GRAPH_URL)

mainnet_pool_snapshots_now = get_balancer_pool_snapshots(mainnet_block, MAINNET_BALANCER_GRAPH_URL)
mainnet_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(mainnet_block_2_weeks_ago),
                                                                 MAINNET_BALANCER_GRAPH_URL)

polygon_pool_snapshots_now = get_balancer_pool_snapshots(poly_block, POLYGON_BALANCER_GRAPH_URL)
polygon_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(poly_block_2_weeks_ago),
                                                                 POLYGON_BALANCER_GRAPH_URL)

## Extract fee data from CORE pools:


In [4]:
from collections import defaultdict
from decimal import Decimal
from notebooks import fetch_token_price_balgql


def collect_fee_info(pools: list[str], chain: str, pools_now: list[dict], pools_shifted: list[dict],
                     start_date: datetime.date) -> tuple[dict, dict]:
    # Iterate through snapshots now and 2 weeks ago and extract fee data, by subtracting today's fee data from 2 weeks ago
    # and then summing across all pools
    fees = {}
    token_fees = defaultdict(list)
    for pool in pools:
        current_fees_snapshots = [x for x in pools_now if x['pool']['id'] == pool]
        current_fees_snapshots.sort(key=lambda x: x['timestamp'], reverse=True)
        fees_2_weeks_ago = [x for x in pools_shifted if x['pool']['id'] == pool]
        fees_2_weeks_ago.sort(key=lambda x: x['timestamp'], reverse=True)
        # Take first element of list, which is the most recent snapshot
        if not current_fees_snapshots or not fees_2_weeks_ago:
            continue
        pool_snapshot_now = current_fees_snapshots[0]
        pool_snapshot_2_weeks_ago = fees_2_weeks_ago[0]
        # Calculate fees
        pool_fee = float(pool_snapshot_now['protocolFee']) - float(pool_snapshot_2_weeks_ago['protocolFee'])
        pool_swap_fee = float(pool_snapshot_now['swapFees']) - float(pool_snapshot_2_weeks_ago['swapFees'])
        # Now we need to collect token fee info. Let's start with BPT tokens, which is Balancer pool token. Notice,
        # That totalProtocolFeePaidInBPT can be null, so we need to check for that
        bpt_token_fee = 0
        bpt_price_usd = arb_bpt_twap_prices[pool] if chain == 'arbitrum' else mainnet_bpt_twap_prices[
            pool] if chain == 'mainnet' else polygon_bpt_twap_prices[pool]
        if bpt_price_usd is None:
            bpt_price_usd = 0
        if pool_snapshot_now['pool']['totalProtocolFeePaidInBPT'] is not None and pool_snapshot_2_weeks_ago['pool'][
            'totalProtocolFeePaidInBPT'] is not None:
            bpt_token_fee = float(pool_snapshot_now['pool']['totalProtocolFeePaidInBPT']) - float(
                pool_snapshot_2_weeks_ago['pool']['totalProtocolFeePaidInBPT'])
            token_fees[pool_snapshot_now['pool']['symbol']].append({
                'token': pool_snapshot_now['pool']['symbol'],
                'token_fee': bpt_token_fee,
                'token_price': bpt_price_usd,
                'token_fee_in_usd': Decimal(bpt_token_fee) * bpt_price_usd,
                'token_addr': pool_snapshot_now['pool']['address'],
                'time_from': arb_datetime_2_weeks_ago,
                'time_to': arb_datetime_now,
                'chain': chain,
            })
        # Now collect fee info about fees paid in pool tokens. Pool tokens fee info is in pool.tokens dictionary. This will be separate dictionary
        else:
            bpt_price_usd = 0
            for token_data in pool_snapshot_now['pool']['tokens']:
                token_data_2_weeks_ago = \
                    [t for t in pool_snapshot_2_weeks_ago['pool']['tokens'] if t['address'] == token_data['address']][0]
                token_fee = float(token_data.get('paidProtocolFees', None)) - float(
                    token_data_2_weeks_ago.get('paidProtocolFees', None))
                # Get twap token price from CoinGecko
                token_price = fetch_token_price_balgql(token_data['address'], chain, start_date)
                token_fees[pool_snapshot_now['pool']['symbol']].append({
                    'token': token_data['symbol'],
                    'token_fee': token_fee,
                    'token_price': token_price,
                    'token_fee_in_usd': Decimal(token_fee) * token_price if token_price is not None else 0,
                    'token_addr': token_data['address'],
                    'time_from': arb_datetime_2_weeks_ago,
                    'time_to': arb_datetime_now,
                    'chain': chain,
                })
        # Calculate non-BPT fees in USD
        fees[pool_snapshot_now['pool']['symbol']] = {
            'pool_fee': round(pool_fee, 2),
            'swap_fee': round(pool_swap_fee, 2),
            'bpt_token_fee': round(bpt_token_fee, 2),
            # Get fee in USD by multiplying bpt_token_fee by price of BPT token taken from twap_bpt_price
            'bpt_token_fee_in_usd': round(Decimal(bpt_token_fee) * bpt_price_usd, 2),
            'token_fees_in_usd': round(sum([x['token_fee_in_usd'] for x in
                                      token_fees[pool_snapshot_now['pool']['symbol']]]) if bpt_price_usd == 0 else 0, 2),
            'time_from': arb_datetime_2_weeks_ago,
            'time_to': arb_datetime_now,
            'chain': chain,
            'token_fees': token_fees[pool_snapshot_now['pool']['symbol']],
        }
    return fees


arb_fees = collect_fee_info(ARB_CORE_POOLS, 'arbitrum', arbi_pool_snapshots_now,
                            arbi_pool_snapshots_2_weeks_ago, arb_datetime_now.date())
mainnet_fees = collect_fee_info(MAINNET_CORE_POOLS, 'mainnet', mainnet_pool_snapshots_now,
                                mainnet_pool_snapshots_2_weeks_ago, mainnet_datetime_now.date())
polygon_fees = collect_fee_info(POLYGON_CORE_POOLS, 'polygon', polygon_pool_snapshots_now,
                                polygon_pool_snapshots_2_weeks_ago, poly_datetime_now.date())
# Convert to dataframe, sort by chain and pool fee
joint_fees = {**arb_fees, **mainnet_fees, **polygon_fees}
joint_fees_df = pd.DataFrame.from_dict(joint_fees, orient='index')

In [5]:
# Remove `token_fees` field from dataframe, as it's too big
joint_fees_df_copy = joint_fees_df.drop(columns=['token_fees'])
# Display all rows in dataframe
pd.set_option('display.max_rows', 1000)
joint_fees_df_copy.sort_values(by=['chain', 'pool_fee'], ascending=False)

,pool_fee,swap_fee,bpt_token_fee,bpt_token_fee_in_usd,token_fees_in_usd,time_from,time_to,chain
MaticX-bb-a-WMATIC-BPT,2636.89,286.01,3882.48,2547.47,0,2023-08-04 03:00:00,2023-08-18 03:00:00,polygon
stMATIC-bb-a-WMATIC-BPT,2096.42,88.94,0.00,0.00,0.00,2023-08-04 03:00:00,2023-08-18 03:00:00,polygon
wstETH-bb-a-WETH-BPT,0.00,0.02,0.00,0.00,0.00,2023-08-04 03:00:00,2023-08-18 03:00:00,polygon
bb-am-usd,0.00,0.00,0.00,0.00,0,2023-08-04 03:00:00,2023-08-18 03:00:00,polygon
wstETH-rETH-sfrxETH-BPT,29878.92,4302.87,15.95,29194.40,0,2023-08-04 03:00:00,2023-08-18 03:00:00,mainnet
B-rETH-STABLE,27610.34,16777.03,0.00,0.00,27055.45,2023-08-04 03:00:00,2023-08-18 03:00:00,mainnet
50STG-50bbaUSD,16308.63,27735.48,20756.67,16559.10,0,2023-08-04 03:00:00,2023-08-18 03:00:00,mainnet
50rETH-50BADGER,12131.48,12205.55,174.04,11638.35,0,2023-08-04 03:00:00,2023-08-18 03:00:00,mainnet
swETH-bb-a-WETH-BPT,7081.62,34.78,3.80,6922.79,0,2023-08-04 03:00:00,2023-08-18 03:00:00,mainnet
ankrETH/wstETH,6092.63,869.23,3.28,6036.83,0,2023-08-04 03:00:00,2023-08-18 03:00:00,mainnet


## Now let's calculate bribes paid to the pools

In [12]:
from notebooks import calculate_aura_vebal_share

aura_vebal_share = calculate_aura_vebal_share(eth_web3, mainnet_block)

# Bribes are split per chain and per pool, with each pool getting a share of the bribe proportional to its share of fees
# paid by all pools on that chain. For example, if pool A paid 10% of all fees on Arbitrum, it will get 10% of the bribes. That 10% will be distributed between aura and vebal, proportional to their share of the bribe.
FEE = Decimal(0.5)  # 50% goes to fees
# Let's calculate share of fees paid by each pool on each chain
def calc_and_split_bribes(fees: Dict, chain: str) -> Dict[str, Dict]:
    pool_bribs = {}
    for pool, data in fees.items():
        pool_fees = data['bpt_token_fee_in_usd'] + data['token_fees_in_usd']
        # Split fees between aura and bal fees
        pool_bribs[pool] = {
            "chain": chain,
            "aura_bribes": pool_fees * FEE * aura_vebal_share,
            "bal_bribes": pool_fees * FEE * (1 - aura_vebal_share),
            "pool_total": pool_fees,
            "fees_taken": pool_fees * FEE,
        }
    return pool_bribs
mainnet_bribes = calc_and_split_bribes(mainnet_fees, 'mainnet')
arb_bribes = calc_and_split_bribes(arb_fees, 'arbitrum')
polygon_bribes = calc_and_split_bribes(polygon_fees, 'polygon')
# Convert to dataframe
joint_bribes_data = {**arb_bribes, **mainnet_bribes, **polygon_bribes}
# Sort by chain:
joint_bribes_data = {k: v for k, v in sorted(joint_bribes_data.items(), key=lambda item: item[1]['chain'])}
joint_bribes_df = pd.DataFrame.from_dict(joint_bribes_data, orient='index')
# Sort by chain
# Dump into csv and prefix with dates
joint_bribes_df.to_csv(f'../data/bribs/bribes_{arb_datetime_2_weeks_ago.date()}_{arb_datetime_now.date()}.csv')
joint_bribes_df

,chain,aura_bribes,bal_bribes,pool_total,fees_taken
rETH-bb-a-WETH-BPT,arbitrum,226.6925211286357966876202621,386.2374788713642033123797379,1225.86,612.930
B-wstETH-WETH-Stable,arbitrum,908.4732464739796115359320646,1547.851753526020388464067935,4912.65,2456.325
wstETH/rETH/cbETH,arbitrum,0E-31,0E-31,0.00,0.000
bb-a-USD,arbitrum,617.2325514103509732238083090,1051.637448589649026776191691,3337.74,1668.870
RDNT-WETH,arbitrum,3153.248106022954846511882491,5372.486893977045153488117509,17051.47,8525.735
bbaUSDC/bbaUSDCe,arbitrum,58.64166003874969204281994789,99.91333996125030795718005211,317.11,158.555
B-rETH-STABLE,mainnet,5003.237050535745815584223011,8524.487949464254184415776989,27055.45,13527.725
swETH-bb-a-WETH-BPT,mainnet,1280.198977325394912103413664,2181.196022674605087896586336,6922.79,3461.395
cbETH-bb-a-WETH-BPT,mainnet,0E-31,0E-31,0.00,0.000
wstETH-rETH-sfrxETH-BPT,mainnet,5398.783008531027117955607476,9198.416991468972882044392524,29194.40,14597.200
